In [1]:
import numpy as np
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Qiskit imports
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, execute
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.algorithms import QSVC
from qiskit.utils import QuantumInstance
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options

# Load the wine dataset
wine = load_wine()
X = wine.data
y = wine.target

# We'll focus on binary classification between first two classes
X = X[y < 2]
y = y[y < 2]

# Scale the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reduce dimensionality to 2 features for simplicity
# We'll use alcohol content and malic acid
X = X[:, [0, 1]]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Dataset shapes:")
print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")


ImportError: cannot import name 'Aer' from 'qiskit' (c:\Users\leqian\anaconda3\envs\qiskit\Lib\site-packages\qiskit\__init__.py)

In [ ]:
# Initialize the IBM Quantum service
# You'll need to replace this with your IBM Quantum API token
service = QiskitRuntimeService(channel="ibm_quantum")

# Get the least busy backend
backend = service.least_busy(operational=True, simulator=False)
print(f"Using backend: {backend.name}")

# Set up the feature map circuit
feature_map = ZZFeatureMap(feature_dimension=2, reps=2)

# Create quantum instance with the selected backend
options = Options()
options.optimization_level = 1
options.resilience_level = 1

# Initialize QSVC
qsvc = QSVC(
    quantum_kernel=feature_map,
    quantum_instance=QuantumInstance(backend=Aer.get_backend('qasm_simulator'))
)


In [ ]:
# Train the model
print("Training the quantum SVM...")
qsvc.fit(X_train, y_train)

# Make predictions
y_pred = qsvc.predict(X_test)

# Calculate accuracy
accuracy = np.mean(y_pred == y_test)
print(f"\nTest accuracy: {accuracy:.3f}")

# Print classification report
from sklearn.metrics import classification_report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


In [ ]:
import matplotlib.pyplot as plt

# Create a mesh grid
x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                     np.arange(y_min, y_max, 0.02))

# Make predictions for each point in the mesh
Z = qsvc.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot the decision boundary
plt.figure(figsize=(10, 8))
plt.contourf(xx, yy, Z, alpha=0.4)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, alpha=0.8)
plt.xlabel('Alcohol Content')
plt.ylabel('Malic Acid')
plt.title('Quantum SVM Decision Boundary on Wine Dataset')
plt.colorbar()
plt.show()
